In [26]:
import pandas as pd
import sqlite3

SQL

**Two tables:**
- Invoices_table - Invoice_Id, User_id, Sent_at, Amount 
- Users_table - User_Id, Business_category, Created_at

In [27]:
invoices_data = {
    'Invoice_Id': [1, 2, 3, 4, 5],
    'User_Id': [1, 2, 1, 3, 2],
    'Sent_at': ['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05'],
    'Amount': [100.00, 200.00, 300.00, 400.00, 500.00]
}

users_data = {
    'User_Id': [1, 2, 3, 4, 5],
    'Business_category': ['Retail', 'Hospitality', 'Technology', 'Education', 'Healthcare'],
    'Created_at': ['2022-01-01', '2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05']
}

invoices_df = pd.DataFrame(invoices_data)
users_df = pd.DataFrame(users_data)

In [28]:
# create connection and convert dataframes to sql tables
cnn = sqlite3.connect('block_ds_intern.db')
invoices_df.to_sql('invoices', cnn, index=False)
users_df.to_sql('users', cnn, index=False)

5

In [29]:
# load sql module to iPython
%load_ext sql
%sql sqlite:///block_ds_intern.db

In [35]:
# cell magic to run sql queries
%%sql 
SELECT * FROM invoices

 * sqlite:///block_ds_intern.db
Done.


Invoice_Id,User_Id,Sent_at,Amount
1,1,2022-01-01,100.0
2,2,2022-01-02,200.0
3,1,2022-01-03,300.0
4,3,2022-01-04,400.0
5,2,2022-01-05,500.0


1) Top 10 senders who sent the most invoice

In [37]:
%%sql
SELECT User_Id, COUNT(*) AS num_invoices
FROM invoices
GROUP BY User_Id
ORDER BY num_invoices DESC
LIMIT 10;

 * sqlite:///block_ds_intern.db
Done.


User_Id,num_invoices
2,2
1,2
3,1


2) The most recent invoice details per sender

In [40]:
%%sql
SELECT i.*
FROM invoices i
INNER JOIN (
    SELECT User_Id, MAX(Sent_at) AS max_sent_at
    FROM invoices
    GROUP BY User_Id
) max_i
ON i.User_Id = max_i.User_Id AND i.Sent_at = max_i.max_sent_at

 * sqlite:///block_ds_intern.db
Done.


Invoice_Id,User_Id,Sent_at,Amount
3,1,2022-01-03,300.0
4,3,2022-01-04,400.0
5,2,2022-01-05,500.0


Same thing using CTE (Common Table Expression) and window function

In [44]:
%%sql
SELECT Invoice_Id, User_Id, Sent_at, Amount
FROM (
    SELECT Invoice_Id, User_Id, Sent_at, Amount,
           ROW_NUMBER() OVER (PARTITION BY User_Id ORDER BY Sent_at DESC) AS rn
    FROM invoices
)
WHERE rn = 1
ORDER BY User_Id

 * sqlite:///block_ds_intern.db
Done.


Invoice_Id,User_Id,Sent_at,Amount
3,1,2022-01-03,300.0
5,2,2022-01-05,500.0
4,3,2022-01-04,400.0


3) How many users in invoices table but not in users table?

In [45]:
%%sql
SELECT COUNT(DISTINCT Invoice_User.User_id) AS num_users
FROM invoices AS Invoice_User
LEFT JOIN users ON Invoice_User.User_id = users.User_Id
WHERE users.User_Id IS NULL;

 * sqlite:///block_ds_intern.db
Done.


num_users
0


4) How many users in users table but not in invoice table?

In [46]:
%%sql
SELECT COUNT(DISTINCT users.User_Id) AS num_users
FROM users
LEFT JOIN invoices AS Invoice_User ON users.User_Id = Invoice_User.User_id
WHERE Invoice_User.Invoice_Id IS NULL;

 * sqlite:///block_ds_intern.db
Done.


num_users
2


5) Total amount, total senders, total invoices by each category

In [47]:
%%sql
SELECT 
    users.Business_category, 
    COUNT(DISTINCT invoices.User_id) AS num_senders,
    COUNT(DISTINCT invoices.Invoice_Id) AS num_invoices,
    SUM(invoices.Amount) AS total_amount
FROM 
    users 
    JOIN invoices ON users.User_Id = invoices.User_id
GROUP BY 
    users.Business_category;

 * sqlite:///block_ds_intern.db
Done.


Business_category,num_senders,num_invoices,total_amount
Hospitality,1,2,700.0
Retail,1,2,400.0
Technology,1,1,400.0


6) For top 10 distinct users, what are the details of their most recent invoice?

In [54]:
%%sql
WITH top_users AS (
    SELECT DISTINCT User_Id, SUM(Amount) AS total_amount
    FROM invoices
    ORDER BY total_amount DESC
    LIMIT 10
), most_recent_invoices AS (
    SELECT Invoice_Id, User_id, Sent_at, Amount,
           ROW_NUMBER() OVER (PARTITION BY User_id ORDER BY Sent_at DESC) AS rn
    FROM invoices
    WHERE User_id IN (SELECT User_Id FROM top_users)
)
SELECT u.*, i.*
FROM most_recent_invoices i
INNER JOIN users u ON i.User_id = u.User_Id
WHERE i.rn = 1

 * sqlite:///block_ds_intern.db
Done.


User_Id,Business_category,Created_at,Invoice_Id,User_id,Sent_at,Amount,rn
1,Retail,2022-01-01,3,1,2022-01-03,300.0,1


Python

In [2]:
Mega = """
FeatureA: tag1, tag2
FeatureB: tag3, tag4
FeatureC: tag3, tag4
FeatureD: tag1, tag3
FeatureE: tag1
"""

1. Parse Mega and store it into a data structure

In [23]:
# Split the string into lines and remove any empty lines
lines = [line.strip() for line in Mega.split('\n') if line.strip()]
# Split each line into a feature and its associated tags
features = [line.split(': ') for line in lines]
# Create a dictionary with the feature as the key and its tags as the value
data_dict = {feature[0]: feature[1].split(', ') for feature in features}
# find all tag mentioned in the data
tags = set()
for feature in data_dict:
    for tag in data_dict[feature]:
        tags.add(tag)
tags_list = list(tags)

In [24]:
# create a dataframe with the tags as columns
df = pd.DataFrame(columns=tags_list)
# fill the dataframe with the features
for feature in data_dict:
    df.loc[feature] = 0 #auto-generated index
    for tag in data_dict[feature]:
        df.loc[feature, tag] = 1
# fill the empty cells with empty string
df = df.fillna('')

2. Find out the features that include some tag

In [15]:
# find out the features that include tag1
df[df['tag1'] == 1].index.tolist()

['FeatureA', 'FeatureD', 'FeatureE']

3. Find out the features that include tag 1 AND tag 2. 

In [16]:
# Find out the features that include tag 1 AND tag 2
df[(df['tag1'] == 1) & (df['tag2'] == 1)].index.tolist()

['FeatureA']

4. Find out the features that include tab A or tag B

In [17]:
# find ou tthe features that include tag 1 OR tag 2
df[(df['tag1'] == 1) | (df['tag2'] == 1)].index.tolist()

['FeatureA', 'FeatureD', 'FeatureE']

5. Find out the pairs of features that share the same tag

In [18]:
# Find out the pairs of features that share the same tag
for tag in tags_list:
    for feature1 in df.index.tolist():
        for feature2 in df.index.tolist():
            if (feature1 != feature2) and (df.loc[feature1, tag] == 1) and (df.loc[feature2, tag] == 1):
                print(feature1, feature2, tag)

FeatureB FeatureC tag3
FeatureB FeatureD tag3
FeatureC FeatureB tag3
FeatureC FeatureD tag3
FeatureD FeatureB tag3
FeatureD FeatureC tag3
FeatureB FeatureC tag4
FeatureC FeatureB tag4
FeatureA FeatureD tag1
FeatureA FeatureE tag1
FeatureD FeatureA tag1
FeatureD FeatureE tag1
FeatureE FeatureA tag1
FeatureE FeatureD tag1


In [59]:
data = """
A: string splittable drop
B: string splittable
C: string categorical
D: string categorical
E: numeric
F: numeric dollars
G: numeric
H: numeric
I: string character
"""

1) write a function ot pase this metadata string and put it into a python function

In [76]:
# write a function ot pase this metadata string and put it into a python function
def parse_metadata(data):
    lines = [line.strip() for line in data.split('\n') if line.strip()]
    features = [line.split(': ') for line in lines]
    data_dict = {feature[0]: feature[1].split(' ') for feature in features}
    # find all tag mentioned in the data
    tags = set()
    for feature in data_dict:
        for tag in data_dict[feature]:
            tags.add(tag)
    tag_list = list(tags)
    # create a dataframe with the tags as columns
    df = pd.DataFrame(columns=tag_list)
    # fill the dataframe with the features
    for feature in data_dict:
        df.loc[feature] = 0 #auto-generated index
        for tag in data_dict[feature]:
            df.loc[feature, tag] = 1
    # fill the empty cells with empty string
    df = df.fillna('')
    return df

# test
df = parse_metadata(data)
df.head()

,character,string,categorical,dollars,drop,splittable,numeric
A,0,1,0,0,1,1,0
B,0,1,0,0,0,1,0
C,0,1,1,0,0,0,0
D,0,1,1,0,0,0,0
E,0,0,0,0,0,0,1


2) write a function that checks which features have a specific tag. What are all the features tagged 'numeric'?

In [77]:
# write a function that checks which features have a specific tag. What are all the features tagged 'numeric'?
def check_tag(df, tag):
    return df[df[tag] == 1].index.tolist()

# test
check_tag(df, 'numeric')

['E', 'F', 'G', 'H']

3) write a funciton that checks which features have N specific tags. What are all the featured tagged 'string' and 'categorical'?

In [82]:
# write a funciton that checks which features have N specific tags. What are all the featured tagged 'string' and 'categorical'?
def check_tags(df, tags):
    condition = df[tags[0]] == 1
    for tag in tags[1:]:
        condition = condition & (df[tag] == 1)
    return df[condition].index.tolist()

# test
check_tags(df, ['string', 'categorical']) 

A     True
B     True
C     True
D     True
E    False
F    False
G    False
H    False
I     True
Name: string, dtype: bool


['C', 'D']